我们使用自己部署的glm2-6b模型和glm-4，仅使用prompt完成一些nlp人物（Prompt 实战）

In [1]:
# chatglm2-6b
import requests
from pprint import pprint

url = "http://localhost:8000/v1/chat/completions"

def get_completion_glm2_6b(prompt, system_prompt=None, temperature=0, top_p=1.0, max_length=8192, stream=False):
    request = {
                'model': 'chatglm2-6b',
                'messages': [{"role": "system", "content": system_prompt}, {"role": "user", "content": prompt}],
                "temperature": temperature,
                "top_p": top_p,
                'max_length': max_length,
                'stream': stream
            }
    response = requests.post(url, json=request).json()

    return response["choices"][0]["message"]['content']

In [2]:
# glm-4
import openai
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

API_KEY = os.getenv("API_KEY")

client = OpenAI(api_key=API_KEY, base_url='https://api.zhizengzeng.com/v1')

def get_completion_glm4(prompt, system_prompt=None, model="glm-4"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": prompt}],
        temperature=0,
    )
    return response.choices[0].message.content # 响应体中提取content

# Task1: text classfication

In [3]:
system_prompt = """
你是一个文本实体识别和分类助手，你可以识别出每段文本中的主体，并判断这是一个什么类型的物体。<>中的内容是两个例子：

<
Q:《航海王》（ONE PIECE）又称《海贼王》，是日本漫画家尾田荣一郎创作的少年漫画作品，于1997年07月22日在集英社《周刊少年Jump》开始连载。
A:《航海王》，类型为：漫画
>
<
Q：日本国（Japan），简称日本，首都东京。是位于太平洋西岸由东北向西南延伸的弧形岛国、美国忠实盟友。陆地面积约37.8万平方公里，包括北海道、本州、四国、九州四个大岛和其它6800多个小岛屿。由1都、1道、2府和43县组成。截至2025年4月，总人口为12340.0万人，是除美国外唯一人口过亿的发达国家，主要民族为大和族，通用日语，主要宗教为神道教和佛教
A：日本，类型为：国家
>
"""

In [4]:
prompt = """
请帮我识别出下面五个句子中，每一句话描述的是一个什么类型的物体。

1. 加拿大（英语/法语：Canada），首都渥太华，位于北美洲北部。东临大西洋，西濒太平洋，西北部邻美国阿拉斯加州，南接美国本土，北靠北冰洋。气候大部分为亚寒带针叶林气候和湿润大陆性气候，北部极地区域为极地长寒气候。

2. 《琅琊榜》是由山东影视传媒集团、山东影视制作有限公司、北京儒意欣欣影业投资有限公司、北京和颂天地影视文化有限公司、北京圣基影业有限公司、东阳正午阳光影视有限公司联合出品，由孔笙、李雪执导，胡歌、刘涛、王凯、黄维德、陈龙、吴磊、高鑫等主演的古装剧。

3. 《满江红》是由张艺谋执导，沈腾、易烊千玺、张译、雷佳音、岳云鹏、王佳怡领衔主演，潘斌龙、余皑磊主演，郭京飞、欧豪友情出演，魏翔、张弛、黄炎特别出演，许静雅、蒋鹏宇、林博洋、飞凡、任思诺、陈永胜出演的悬疑喜剧电影。

4. 布宜诺斯艾利斯（Buenos Aires，华人常简称为布宜诺斯）是阿根廷共和国（the Republic of Argentina，República Argentina）的首都和最大城市，位于拉普拉塔河南岸、南美洲东南部、河对岸为乌拉圭东岸共和国。

5. 张译（原名张毅），1978年2月17日出生于黑龙江省哈尔滨市，中国内地男演员。1997年至2006年服役于北京军区政治部战友话剧团。2006年，主演军事励志题材电视剧《士兵突击》。

"""

In [6]:
response = get_completion_glm2_6b(prompt, system_prompt)
print(response)

1. 加拿大，类型为：国家
2. 《琅琊榜》，类型为：电视剧
3. 《满江红》，类型为：电影
4. 布宜诺斯艾利斯，类型为：城市
5. 张译，类型为：演员


In [7]:
response = get_completion_glm4(prompt, system_prompt)
print(response)

1. 加拿大，类型为：国家
2. 《琅琊榜》，类型为：电视剧
3. 《满江红》，类型为：电影
4. 布宜诺斯艾利斯，类型为：城市
5. 张译，类型为：人物（具体为演员）


# Task2: text matching

In [8]:
system_prompt = """
你是一个文本匹配助手，你可以识别出一个文本对中，两个句子描述的内容是否相似。<>中的内容是一个输出格式的例子：

<
Q:('动漫《Fate/zero》中，兰斯洛特死于阿尔托莉雅之手', '兰斯洛特是亚瑟王传说中最著名的圆桌骑士之一')
A:不相似
>
"""

In [9]:
prompt = """
下面是 3 个文本对，请帮我判断哪几对描述的是相似的内容，不要做多余的解释

1. ('如何修改头像', '可以通过上传图片修改头像吗')
2. ('王者荣耀司马懿连招', '王者荣耀司马懿有什么技巧')
3. ('王者荣耀司马懿连招', '历史上司马懿真的被诸葛亮空城计骗了吗')

"""

In [10]:
response = get_completion_glm2_6b(prompt, system_prompt)
print(response)

1. ('如何修改头像', '可以通过上传图片修改头像吗') 不相似
2. ('王者荣耀司马懿连招', '王者荣耀司马懿有什么技巧') 相似
3. ('王者荣耀司马懿连招', '历史上司马懿真的被诸葛亮空城计骗了吗') 不相似


In [12]:
response = get_completion_glm4(prompt, system_prompt)
print(response)

<
Q:('如何修改头像', '可以通过上传图片修改头像吗')
A:相似

Q:('王者荣耀司马懿连招', '王者荣耀司马懿有什么技巧')
A:相似

Q:('王者荣耀司马懿连招', '历史上司马懿真的被诸葛亮空城计骗了吗')
A:不相似
>


# 3 Information Extraction

In [13]:
system_prompt = """
你是一个信息抽取助手，你将对一个长文本进行信息抽取，抽取的实体数据格式如schema所示：

schema = {
  '人物': ['姓名', '性别', '出生日期', '出生地点', '职业', '获得奖项', '实体类型'],
  '书籍': ['作者', '类型', '发行时间', '定价', '实体类型'],
  '电视剧': ['导演', '演员', '题材', '出品方', '实体类型']
  '电影': ['导演', '演员', '题材', '出品方', '实体类型']
}

采用json输出，如果原文没有，则对应字段的取值为“原文中未提及”，<>中的内容是一个案例

<
Q:《笑猫日记》是作家杨红樱创作的儿童文学系列小说，于2006年5月首次开始出版，共30种。该系列小说主要讲述了笑猫与其家人、朋友的经历、冒险。小说以笑猫为故事的讲述者，用日记体的写作形式，创造了动物与孩子共同参与、幻想与现实相结合的多层次的文学世界。
A:'书籍': [
    {
      'content': '《笑猫日记》是作家杨红樱创作的儿童文学系列小说，于2006年5月首次开始出版，共30种。',
      'answers': {
      '作者': ['杨红樱'],
      '类型': ['儿童文学系列小说'],
      '发行时间': ['2006年5月'],
      '定价': ['原文中未提及']
      }
    }
  ]
>
"""

In [14]:
prompt = """
请帮我识别下面这 3 个句子中的实体信息，采用json输出：

1.《琅琊榜》是由山东影视传媒集团、山东影视制作有限公司、北京儒意欣欣影业投资有限公司、北京和颂天地影视文化有限公司、北京圣基影业有限公司、东阳正午阳光影视有限公司联合出品，由孔笙、李雪执导，胡歌、刘涛、王凯、黄维德、陈龙、吴磊、高鑫等主演的古装剧。

2.《满江红》是由张艺谋执导，沈腾、易烊千玺、张译、雷佳音、岳云鹏、王佳怡领衔主演，潘斌龙、余皑磊主演，郭京飞、欧豪友情出演，魏翔、张弛、黄炎特别出演，许静雅、蒋鹏宇、林博洋、飞凡、任思诺、陈永胜出演的悬疑喜剧电影。

3. 张译（原名张毅），1978年2月17日出生于黑龙江省哈尔滨市，中国内地男演员。1997年至2006年服役于北京军区政治部战友话剧团。2006年，主演军事励志题材电视剧《士兵突击》。
"""

In [15]:
response = get_completion_glm2_6b(prompt, system_prompt)
print(response)

1.《琅琊榜》是由山东影视传媒集团、山东影视制作有限公司、北京儒意欣欣影业投资有限公司、北京和颂天地影视文化有限公司、北京圣基影业有限公司、东阳正午阳光影视有限公司联合出品，由孔笙、李雪执导，胡歌、刘涛、王凯、黄维德、陈龙、吴磊、高鑫等主演的古装剧。

{
  '人物': [
    {
      '姓名': '胡歌',
      '性别': '男',
      '出生日期': '1982年9月28日',
      '出生地点': '上海',
      '职业': '演员',
      '获得奖项': [
        {
          '奖项': '金球奖',
          '年份': '2022年'
        }
      ],
      '实体类型': '演员'
    },
    {
      '姓名': '刘涛',
      '性别': '女',
      '出生日期': '1978年1月11日',
      '出生地点': '北京',
      '职业': '演员',
      '获得奖项': [
        {
          '奖项': '金熊奖',
          '年份': '2022年'
        }
      ],
      '实体类型': '演员'
    },
    {
      '姓名': '王凯',
      '性别': '男',
      '出生日期': '1985年7月21日',
      '出生地点': '北京市',
      '职业': '演员',
      '获得奖项': [
        {
          '奖项': '金狮奖',
          '年份': '2022年'
        }
      ],
      '实体类型': '演员'
    },
    {
      '姓名': '黄维德',
      '性别': '男',
      '出生日期': '1977年10月28日',
      '出生地点': '北京市',
      '职业': '演员',
      '获得奖项': [
        {
          '奖项': '金球奖',
          '年份': '2022年'
  

In [16]:
response = get_completion_glm4(prompt, system_prompt)
print(response)

以下是按照提供的schema格式抽取的实体信息：

```json
{
  "电视剧": [
    {
      "content": "《琅琊榜》是由山东影视传媒集团、山东影视制作有限公司、北京儒意欣欣影业投资有限公司、北京和颂天地影视文化有限公司、北京圣基影业有限公司、东阳正午阳光影视有限公司联合出品，由孔笙、李雪执导，胡歌、刘涛、王凯、黄维德、陈龙、吴磊、高鑫等主演的古装剧。",
      "answers": {
        "导演": ["孔笙", "李雪"],
        "演员": ["胡歌", "刘涛", "王凯", "黄维德", "陈龙", "吴磊", "高鑫"],
        "出品方": ["山东影视传媒集团", "山东影视制作有限公司", "北京儒意欣欣影业投资有限公司", "北京和颂天地影视文化有限公司", "北京圣基影业有限公司", "东阳正午阳光影视有限公司"],
        "题材": ["古装剧"],
        "实体类型": "电视剧"
      }
    }
  ],
  "电影": [
    {
      "content": "《满江红》是由张艺谋执导，沈腾、易烊千玺、张译、雷佳音、岳云鹏、王佳怡领衔主演，潘斌龙、余皑磊主演，郭京飞、欧豪友情出演，魏翔、张弛、黄炎特别出演，许静雅、蒋鹏宇、林博洋、飞凡、任思诺、陈永胜出演的悬疑喜剧电影。",
      "answers": {
        "导演": ["张艺谋"],
        "演员": ["沈腾", "易烊千玺", "张译", "雷佳音", "岳云鹏", "王佳怡", "潘斌龙", "余皑磊", "郭京飞", "欧豪", "魏翔", "张弛", "黄炎", "许静雅", "蒋鹏宇", "林博洋", "飞凡", "任思诺", "陈永胜"],
        "题材": ["悬疑喜剧"],
        "实体类型": "电影"
      }
    }
  ],
  "人物": [
    {
      "content": "张译（原名张毅），1978年2月17日出生于黑龙江省哈尔滨市，中国内地男演员。1997年至2006年服役于北京军区政治部战友话剧团。2006年，主演军事励志题材电视剧《士兵突击》。",
     